<a href="https://colab.research.google.com/github/naku2/smartFactory/blob/main/FastAPI_leenakkyu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [1]:
%%capture
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install fastapi
!pip install uvicorn
!pip install pyngrok
!pip install python-multipart==0.0.5

from google.colab import drive
drive.mount('/content/drive')

## 모델 구동 확인

In [2]:
import tensorflow
from tensorflow import keras
from typing import Union
from fastapi import FastAPI, UploadFile, File
from keras.models import load_model
from starlette.requests import Request
from fastapi import FastAPI, HTTPException, UploadFile, File
from starlette.requests import Request
from starlette.responses import HTMLResponse
from keras.models import load_model
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from PIL import Image
import numpy as np
import io
import shutil
import os

app = FastAPI()

model = load_model('/content/drive/MyDrive/SK/smartFactory/transistor_cnn.h5')

templates = Jinja2Templates(directory="/content/drive/MyDrive/SK/smartFactory")
app.mount("/static", StaticFiles(directory="/content/drive/MyDrive/SK/smartFactory/static/"))
upload_folder = "/content/drive/MyDrive/SK/smartFactory/static/"

image = Image.open('/content/drive/MyDrive/SK/smartFactory/static/big_head.jpeg')
image = image.resize((224, 224)).convert('RGB')
image = np.array(image) / 255.0
image = image.reshape(-1, 224, 224, 3)

pred = model.predict(image)
result = np.argmax(pred)

prediction_success = result
prediction_result = result

1/1 [==============================] - 3s 3s/step


In [3]:
print(result)

0


## API 제작

In [4]:
import tensorflow
from tensorflow import keras
from typing import Union
from fastapi import FastAPI, UploadFile, File
from keras.models import load_model
from starlette.requests import Request
from fastapi import FastAPI, HTTPException, UploadFile, File
from starlette.requests import Request
from starlette.responses import HTMLResponse
from keras.models import load_model
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from PIL import Image
import numpy as np
import io
import shutil
import os

app = FastAPI()

model = load_model('/content/drive/MyDrive/SK/smartFactory/transistor_cnn.h5')

templates = Jinja2Templates(directory="/content/drive/MyDrive/SK/smartFactory")
app.mount("/static", StaticFiles(directory="/content/drive/MyDrive/SK/smartFactory/static/"))
upload_folder = "/content/drive/MyDrive/SK/smartFactory/static/"


@app.get("/", response_class=HTMLResponse)
async def read_root(request: Request):
    return templates.TemplateResponse("main.html", {"request": request})

@app.get("/predict", response_class=HTMLResponse)
async def read_predict(prediction_success: bool = True):
    return templates.TemplateResponse("predict.html", {"prediction_success": prediction_success, "image_path": "/static/image_filename.jpg"})

@app.post("/uploadimage/")
async def upload_image(request: Request, image: UploadFile = File(...)):
    try:
        with open(os.path.join(upload_folder, image.filename), "wb") as buffer:
            shutil.copyfileobj(image.file, buffer)

        image = Image.open(io.BytesIO(await image.read()))
        image = image.resize((224, 224)).convert('RGB')
        image = np.array(image) / 255.0
        image = image.reshape(-1, 224, 224, 3)

        pred = model.predict(image)
        result = np.argmax(pred)

        prediction_success = result
        prediction_result = result

        return templates.TemplateResponse("predict.html", {"request": request, "prediction_success": prediction_success, "prediction_result": prediction_result, "image_path": f"/static/{image.filename}"})

    except Exception as e:
        # 예외 발생 시 에러 메시지를 출력하여 디버깅
        print("Error occurred:", e)
        # 예외가 발생한 경우에도 적절한 응답을 반환하도록 처리
        return templates.TemplateResponse("predict.html", {"request": request, "prediction_success": False, "prediction_result": "Error", "image_path": f"/static/{image.filename}"})



## ngrok 환경설정

In [5]:
from pyngrok import ngrok
import uvicorn
import threading

!ngrok authtoken '2SrfOHr1mTjMqKQmg6UGJ7fHt1m_fbeBZagYob3KbzxjKGXf'

ngrok.kill()

url = ngrok.connect(8000)  # FastAPI의 기본 포트는 8000입니다.
print('Public URL:', url)

def run():
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info", access_log=False)
    config.access_log_file = "log.txt"  # 로그를 파일로 저장하는 설정
    server = uvicorn.Server(config)
    server.run()

# run 함수를 쓰레드로 실행
threading.Thread(target=run).start()


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Public URL: NgrokTunnel: "https://90d5-35-236-210-37.ngrok-free.app" -> "http://localhost:8000"


In [6]:
!lsof -i :8000

INFO:     Started server process [225]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


COMMAND PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3 225 root   53u  IPv4  47448      0t0  TCP *:8000 (LISTEN)


In [7]:
!kill -9 387

/bin/bash: line 1: kill: (387) - No such process
